In [ ]:
import anthropic
import os
from groq import Groq

# print(os.getenv("ANTHROPIC_API_KEY"))
claude_client = anthropic.Anthropic(
    # defaults to os.environ.get("ANTHROPIC_API_KEY")
    api_key=os.environ.get("ANTHROPIC_API_KEY"),
)
groq_client = Groq(
        api_key=os.environ.get("GROQ_API_KEY"),
    )


In [ ]:
def translate_into_hebrew(input_text):
    message = claude_client.messages.create(
        model="claude-3-opus-20240229",
        max_tokens=4000,
        temperature=0.0,
        system="translate the input into Hebrew return the translated text",
        messages=[
            {"role": "user", "content": input_text}
        ]
    )
    return message.content

In [ ]:
def translate_into_english(input_text):
    message = claude_client.messages.create(
        model="claude-3-opus-20240229",
        max_tokens=4000,
        temperature=0.0,
        system="translate the input into English return the translated text",
        messages=[
            {"role": "user", "content": input_text}
        ]
    )
    return message.content

In [ ]:
import PyPDF2
def read_pdf_file(file_path):
    reader=PyPDF2.PdfReader(file_path)
    pages=[]
    for page in reader.pages:
        pages.append(page.extract_text())
    return pages


In [ ]:
pages=read_pdf_file("cleaning_sop.pdf")

In [ ]:
print(pages[2])

In [21]:
translated_text=[]
for page in pages:
    res=translate_into_english(page)
    translated_text.append(res[0].text)

In [26]:
def summerize_pages(pages):
    res=[]
    for page in pages:
        chat_completion = groq_client.chat.completions.create(
            messages=[
                {
                    "role": "user",
                    "content": "summarize the following text: "+page,
                }
            ],
            # model="mixtral-8x7b-32768"
            model="llama3-70b-8192",
        )
        res.append(chat_completion.choices[0].message.content)
    return res

In [33]:
def summerize_text(text):
    chat_completion = groq_client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": "summarize the following text: "+text,
            }
        ],
        # model="mixtral-8x7b-32768"
        model="llama3-70b-8192",
    )
    return chat_completion.choices[0].message.content

In [27]:
summerized_pages=summerize_pages(translated_text)

In [32]:
#concatenate the summerized pages
final_text=""
for page in summerized_pages:
    final_text+=page+"\n\n"
    
from nltk.tokenize import word_tokenize
print(len(word_tokenize(final_text)))

636


In [34]:
print(summerize_text(final_text))

Here is a summary of the text:

The document outlines the cleaning procedures for Elcam Medical's sites, with the goal of maintaining a clean and controlled environment in accordance with Good Manufacturing Practice (GMP) rules. The procedures cover various areas, including clean rooms, preparation rooms, and public areas, and specify:

* Cleaning schedules for different areas, including daily, twice daily, weekly, and biweekly cleaning
* Use of approved cleaning materials and equipment
* Safety rules during cleaning
* Documentation and monitoring of cleaning procedures

The document is valid from March 21, 2024, to April 25, 2024, and outlines specific procedures for:

* Clean room cleaning
* Preparation room cleaning
* Public area cleaning
* Logistics warehouse and gray packaging area cleaning
* Control and inspection of cleaning
* Cleaning of production baths
* Record keeping

The document is distributed to various personnel, including production managers, logistics managers, and qu